## ResNet

ResNet y VGG comparten la idea de bloques repetitivos, pero difieren en lo que hacen dentro del bloque y en cómo conectan los bloques entre sí.

ResNet se basa en "bloques residuales".
Cada bloque tiene un atajo (“skip connection”) que suma la entrada original del bloque (x) con la salida procesada (F(x)) para tratar de solucionar el problema de desvanecimiento del gradiente.

De esta forma, el gradiente puede fluir hacia atras sin perderse en redes mucho más profundas.

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
from datetime import datetime
from torchmetrics.classification import MulticlassPrecision, MulticlassRecall, MulticlassF1Score

TENSORBOARD_EXP = f"runs/cifar10_cnn_step20_lr0015_{datetime.now().strftime('%Y%m%d-%H%M%S')}"


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

torch.manual_seed(3)

transform = transforms.ToTensor()

train_full = datasets.CIFAR10("./data", train=True, download=True, transform=transform)


#Obtenemos el dataset train completo
loader_train = DataLoader(train_full, batch_size=len(train_full), shuffle=False, drop_last=True)                       
#Obtenemos un batch de datos
imgs, _ = next(iter(loader_train))             
print(imgs.shape) # [50000,3,32,32]

CIFAR10_MEAN = imgs.mean(dim=(0,2,3))
CIFAR10_STD  = imgs.std(dim=(0,2,3))
print(CIFAR10_MEAN, CIFAR10_STD)


In [ ]:

#CIFAR10_MEAN = torch.tensor([0.4914, 0.4822, 0.4465])
#CIFAR10_STD  = torch.tensor([0.2470, 0.2435, 0.2616])



#Creamos el transform para data augmentation y normalización de TRAIN_SET

train_tf = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN.tolist(), CIFAR10_STD.tolist())
    
])

#Creamos el transform para normalización de TRAIN_SET y EVAL sin aug
no_aug = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(CIFAR10_MEAN.tolist(), CIFAR10_STD.tolist())
])

# Re-creamos los datasets con el nuevo transform + uno nuevo para validacion y que no pase por aug
train_full_aug = datasets.CIFAR10(root="./data", train=True, download=True, transform=train_tf)

train_full_no_aug = datasets.CIFAR10(root="./data", train=True, download=True, transform=no_aug)

test_set = datasets.CIFAR10(root="./data", train=False, download=True, transform=no_aug)


#Comprobamos que los datos estan normalizados
check_loader_train_full = DataLoader(train_full_no_aug, batch_size=len(train_full_no_aug), shuffle=False) #Dataset completo para calcular la media y la desviación estándar de los datos ya normalizados
x, _ = next(iter(check_loader_train_full))
mean_check = x.mean(dim=(0, 2, 3))
std_check  = x.std(dim=(0, 2, 3))
print("Mean Appx: 0):", mean_check)
print("Std Appx: 1):", std_check)
assert mean_check.abs().max() < 0.05
assert (std_check - 1).abs().max() < 0.05



#Dividimos el dataset en train y validation para no_aug
train_set, val_set = torch.utils.data.random_split(generator=torch.Generator().manual_seed(3), dataset=train_full_no_aug, lengths=[40000, 10000])

#Dividimos el dataset en train y validation para AUG
train_set_aug, val_set_aug = torch.utils.data.random_split(generator=torch.Generator().manual_seed(3), dataset=train_full_aug, lengths=[40000, 10000])

#Comprobamos que el dataset se ha dividido correctamente
print(len(train_set), len(val_set))
assert train_set.indices == train_set_aug.indices
assert val_set.indices   == val_set_aug.indices


#Nuestros loaders para entrenar, validar y testear, ya normalizados.

#Vamos a mejorar rendimiento de la GPU

NUM_WORKERS = 4

loader_train = DataLoader(train_set_aug, batch_size=128, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True, persistent_workers=True, prefetch_factor=2) #Barajamos porque es train y mejora la generalización
loader_val = DataLoader(val_set, batch_size=256, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True, persistent_workers=True, prefetch_factor=2)
loader_test = DataLoader(test_set, batch_size=256, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True, persistent_workers=True, prefetch_factor=2)






